#**Pre-request**

##Mount google drive


In [1]:
### **Mount** Google Drive
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

##Install pakages


In [ ]:
#Install pakages
project_path = "/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/"

#%pip install -q -r /content/drive/MyDrive/Sem-6/coding/github/fraud_detection/ML_requirements.txt --no-cache-dir
!pip install -q -r "{project_path}requirement/Install/ML_requirements.txt" --no-cache-dir




In [ ]:
%cd $project_path
%ls $project_path

##Import  libs

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import yaml
import logging
import datetime
from google.colab import data_table
data_table.enable_dataframe_formatter()
# Expand Colab’s table display limits
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
from sklearn.metrics import (
    roc_auc_score,
    classification_report,
    recall_score
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, recall_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking, Dropout
from tensorflow.keras.optimizers import Adam
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from sklearn.metrics import roc_auc_score, recall_score, classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, recall_score
import shap


##Config

In [ ]:
import yaml
import os
import logging

logger = logging.getLogger(__name__)

def load_config(config_path="configs/baseline.yaml"):
    """Load YAML config file and expand ${root_path} placeholders."""
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)

    logger.info(f"✅ Loaded config from {config_path}")

    # --- Expand ${root_path} placeholders ---
    root = config.get("root_path", "")

    def expand_paths(obj):
        if isinstance(obj, dict):
            return {k: expand_paths(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [expand_paths(i) for i in obj]
        elif isinstance(obj, str) and "${root_path}" in obj:
            return obj.replace("${root_path}", root)
        else:
            return obj

    config = expand_paths(config)
    return config


#ML Modules

##Snapshot based

###Load snapshots

In [ ]:

config = load_config(os.path.join(project_path, "configs", "baseline.yaml"))

snapshot_path = config['ML']['snapshot_input'] + config['ML']['snapshot_file']
df = pd.read_csv(snapshot_path)

print(f"✅ Loaded snapshot dataset: {df.shape}")



##Split users level

In [ ]:

user_path = config['ML']['Events']['base_path'] + config['ML']['Events']['files']['user']
df_user = pd.read_csv(user_path)
print(f"✅ Loaded transactional user dataset: {df_user.shape}")



# Aggregate to one row per user (max label = 1 if any fraud)
user_labels = df_user.groupby("phone_no_m")["label"].max()
print(f"👥 Unique users for splitting: {len(user_labels)}")

# ==============================================================
# 2️⃣ Create user-level split (stratified, no leakage)
# ==============================================================

fraud_users = user_labels[user_labels == 1].index
normal_users = user_labels[user_labels == 0].index

fraud_train, fraud_test = train_test_split(fraud_users, test_size=0.2, random_state=42)
normal_train, normal_test = train_test_split(normal_users, test_size=0.2, random_state=42)

train_users = set(fraud_train) | set(normal_train)
test_users  = set(fraud_test)  | set(normal_test)

# ==============================================================
# 3️⃣ Save unified split (shared across LSTM / RF / XGB)
# ==============================================================

split_dir = "splits"
os.makedirs(split_dir, exist_ok=True)

pd.DataFrame({"phone_no_m": sorted(train_users)}).to_csv(f"{split_dir}/train_users.csv", index=False)
pd.DataFrame({"phone_no_m": sorted(test_users)}).to_csv(f"{split_dir}/test_users.csv", index=False)

# ==============================================================
# 4️⃣ Summary
# ==============================================================

print("\n👥 Users Summary:")
print(f"   Total : {len(user_labels):,}")
print(f"   Fraud : {len(fraud_users):,} ({len(fraud_users)/len(user_labels)*100:.2f}%)")
print(f"   Normal: {len(normal_users):,} ({len(normal_users)/len(user_labels)*100:.2f}%)")

print("\n📂 Split saved to /splits/:")
print(f"   Train users: {len(train_users)}")
print(f"   Test  users: {len(test_users)}")
print(f"   Fraud ratio train: {len(fraud_train)/len(train_users)*100:.2f}%")
print(f"   Fraud ratio test : {len(fraud_test)/len(test_users)*100:.2f}%")


###drop colums

In [ ]:
def prepare_features(df):
    """
    Selects only the explicitly defined features for model training.
    You control which features are used by editing 'selected_features' below.
    """

    # --- Define selected features manually ---
   # selected_features = [
    #    "window_size", "voc_total_calls", "voc_unique_contacts", "voc_total_duration",
     #  "voc_avg_duration", "voc_max_duration", "voc_std_duration", "voc_active_days",
      # "voc_active_hours", "sms_total_msgs", "sms_unique_contacts", "sms_active_hours",
      # "sms_calltype_ratio", "app_months_active", "app_total_flow", "app_avg_flow",
      # "app_std_flow", "app_unique_apps_mean", "app_unique_apps_max", "user_months_active",
      #  "arpu_mean", "arpu_std", "arpu_max", "idcard_cnt", "snapshot_round"
  # ]
    selected_features = [
        "voc_total_calls", "voc_unique_contacts", "voc_total_duration",
       "voc_avg_duration", "voc_max_duration", "voc_std_duration", "voc_active_days",
      "voc_active_hours", "sms_total_msgs", "sms_unique_contacts", "sms_active_hours",
     "sms_calltype_ratio", "idcard_cnt"
    ]

    # ✅ You can manually remove or comment out features here
    # For example:
    # selected_features = [f for f in selected_features if not (f.startswith("app_") or f.startswith("arpu_"))]

    # --- Keep only existing columns ---
    available = [f for f in selected_features if f in df.columns]
    missing = [f for f in selected_features if f not in df.columns]

    X = df[available].copy()

    print(f"\n📊 Final features used ({len(available)}): {available}")
    if missing:
        print(f"⚠️ Missing columns not found in data: {missing}")

    return X


###Split for snapshot based

In [ ]:


# ==============================================================
# 1️⃣ Load or Create Unified User Split
# ==============================================================

split_dir = "splits"
train_split_file = f"{split_dir}/train_users.csv"
test_split_file  = f"{split_dir}/test_users.csv"

if os.path.exists(train_split_file) and os.path.exists(test_split_file):
    print("📂 Using existing user split from file...")
    train_users = set(pd.read_csv(train_split_file)["phone_no_m"])
    test_users  = set(pd.read_csv(test_split_file)["phone_no_m"])
else:
    print("🆕 Creating new unified user split...")
    os.makedirs(split_dir, exist_ok=True)

    user_labels = df.groupby("phone_no_m")["label"].max()
    fraud_users  = user_labels[user_labels == 1].index
    normal_users = user_labels[user_labels == 0].index

    fraud_train, fraud_test = train_test_split(fraud_users, test_size=0.2, random_state=42)
    normal_train, normal_test = train_test_split(normal_users, test_size=0.2, random_state=42)

    train_users = set(fraud_train) | set(normal_train)
    test_users  = set(fraud_test)  | set(normal_test)

    pd.DataFrame({"phone_no_m": sorted(train_users)}).to_csv(train_split_file, index=False)
    pd.DataFrame({"phone_no_m": sorted(test_users)}).to_csv(test_split_file, index=False)
    print(f"✅ Saved user split to '{split_dir}/'")

print(f"✅ Train users: {len(train_users)} | Test users: {len(test_users)}")

# ==============================================================
# 2️⃣ Apply User Split to Snapshot Data
# ==============================================================

train_df = df[df["phone_no_m"].isin(train_users)]
test_df  = df[df["phone_no_m"].isin(test_users)]

assert len(set(train_df["phone_no_m"]) & set(test_df["phone_no_m"])) == 0, "❌ User leakage detected!"
assert train_df["label"].nunique() == 2, "❌ Training set must contain both classes"
assert test_df["label"].nunique() == 2, "❌ Test set must contain both classes"

print(f"\n👥 User Summary:")
print(f"   Train users: {len(train_users):,}")
print(f"   Test  users: {len(test_users):,}")

print(f"\n📊 Event Split Summary:")
for name, df_part in [("Train", train_df), ("Test", test_df)]:
    total = len(df_part)
    fraud = df_part["label"].sum()
    print(f"   {name:5s} → {total:,} snapshots | Fraud: {fraud/total*100:.2f}% | Normal: {(1 - fraud/total)*100:.2f}%")

# ==============================================================
# 3️⃣ Feature Preparation
# ==============================================================



# Case 1: include app* and arpu* columns
X_train = prepare_features(train_df)
X_test  = prepare_features(test_df)


# Case 2: exclude app* and arpu* columns
# X_train = prepare_features(train_df, use_app_arpu=False)
# X_test  = prepare_features(test_df, use_app_arpu=False)


# Align features (ensure same structure)
X_train, X_test = X_train.align(X_test, join="left", axis=1, fill_value=0)

y_train = train_df["label"].astype(int)
y_test  = test_df["label"].astype(int)

# ==============================================================
# 4️⃣ Full Dataset Scan + Scaling
# ==============================================================

# Combine all snapshots (for robust scaling reference)
all_data = pd.concat([train_df, test_df], axis=0)
print(f"\n📦 Scanning full dataset for scaling — total rows: {len(all_data):,}, columns: {len(all_data.columns)}")

numeric_cols = X_train.columns  # only use columns actually selected for the model
summary = all_data[numeric_cols].describe().T


# Select columns to scale (exclude binary or constants)
scale_cols = [
    c for c in numeric_cols
    if (summary.loc[c, "max"] - summary.loc[c, "min"]) > 5 and summary.loc[c, "max"] > 1
]
scale_cols = [c for c in scale_cols if c in X_train.columns]

print("\n📊 Numeric feature summary (before scaling):")
print(summary[["min", "max", "mean"]].round(2).sort_values("max", ascending=False))

print("\n📏 Selected for scaling (auto-detected based on range):")
print(scale_cols)

# Apply scaling
scaler = RobustScaler().fit(all_data[scale_cols])
X_train[scale_cols] = scaler.transform(X_train[scale_cols])
X_test[scale_cols]  = scaler.transform(X_test[scale_cols])

print("\n✅ Scaled snapshot features successfully.")
print(f"   Scaled columns: {len(scale_cols)} of {X_train.shape[1]} total features.")

# ==============================================================
# 5️⃣ Final Sanity Checks
# ==============================================================

print("\n✅ Feature Matrices Ready:")
print(f"   X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"   X_test : {X_test.shape}, y_test : {y_test.shape}")

print("\n🔒 Consistency Check: ✅ Same users used for all models (LSTM, RF, XGBoost).")


###

###cor

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# --- Snapshot correlation (XGBoost & RF) ---
print("📊 Correlation Matrix — Snapshot Features (XGBoost & RF)")

corr_snapshot = X_train.corr()

plt.figure(figsize=(12,8))
sns.heatmap(corr_snapshot, cmap='coolwarm', annot=False, fmt=".2f", linewidths=0.5)
plt.title("📊 Feature Correlation Heatmap — Snapshot Data (XGBoost & RF)")
plt.show()

# Optional: List highly correlated pairs
threshold = 0.85
corr_pairs = (
    corr_snapshot.where(np.triu(np.ones(corr_snapshot.shape), k=1).astype(bool))
    .stack()
    .reset_index()
)
corr_pairs.columns = ["Feature1", "Feature2", "Correlation"]
high_corr_snapshot = corr_pairs[corr_pairs["Correlation"].abs() > threshold]
display(high_corr_snapshot)


###XGBoost

####Training

In [ ]:

xgb_model  = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    eval_metric='auc',
    scale_pos_weight=2.6,
    min_child_weight=1,
    gamma=0.1
    #--tree_method='gpu_hist',
    #--predictor='gpu_predictor'

)
print("🚀 Training XGBoost...")
xgb_model .fit(X_train, y_train)


###RF

####Train

In [ ]:
# ✅ Train Random Forest in parallel
rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=3,
    random_state=42,
    n_jobs=-1
)

print("🌲 Training Random Forest...")
rf_model.fit(X_train, y_train)





###Test

In [ ]:
from sklearn.metrics import f1_score

snapshot_indices = []
snapshot_metrics_xgb = []
snapshot_metrics_rf = []
recalls_xgb = []
recalls_rf = []
f1s_xgb, f1s_rf = [], []


for snap_idx, group in test_df.groupby('snapshot_index'):
    y_true = group['label']
    if y_true.nunique() < 2:
        continue

    X_snap = prepare_features(group)
    X_snap = X_snap.reindex(columns=X_train.columns, fill_value=0)

    # 🔹 XGBoost
    y_pred_xgb = xgb_model.predict_proba(X_snap)[:, 1]
    auc_xgb = roc_auc_score(y_true, y_pred_xgb)
    rec_xgb = recall_score(y_true, (y_pred_xgb > 0.5).astype(int))
    f1_xgb  = f1_score(y_true, (y_pred_xgb > 0.5).astype(int))


    # 🔹 Random Forest
    y_pred_rf = rf_model.predict_proba(X_snap)[:, 1]
    auc_rf = roc_auc_score(y_true, y_pred_rf)
    rec_rf = recall_score(y_true, (y_pred_rf > 0.5).astype(int))
    f1_rf  = f1_score(y_true, (y_pred_rf > 0.5).astype(int))


    # Append results
    snapshot_indices.append(snap_idx)
    snapshot_metrics_xgb.append(auc_xgb)
    snapshot_metrics_rf.append(auc_rf)
    recalls_xgb.append(rec_xgb)
    recalls_rf.append(rec_rf)
    f1s_xgb.append(f1_xgb)
    f1s_rf.append(f1_rf)


###Report

In [ ]:

def evaluate_model_global(model, X_test, y_test, model_name="Model"):
    """
    Evaluate model on test data and display classification report + confusion matrix.
    """
    # Predict probabilities and labels
    y_pred_prob = model.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_prob > 0.5).astype(int)

    # Classification Report
    print(f"\n📊 Classification Report — {model_name}")
    print(classification_report(y_test, y_pred, digits=4))

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Normal (0)", "Fraud (1)"])
    disp.plot(cmap="Blues")
    plt.title(f"Confusion Matrix — {model_name}")
    plt.grid(False)
    plt.show()

    return y_pred, y_pred_prob, cm

In [ ]:


plt.figure(figsize=(9,6))
plt.plot(snapshot_indices, snapshot_metrics_xgb, 'b-o', label='XGBoost AUC')
plt.plot(snapshot_indices, snapshot_metrics_rf, 'g--o', label='RandomForest AUC')
plt.plot(snapshot_indices, recalls_xgb, 'orange', marker='s', linestyle='--', label='XGBoost Recall')
plt.plot(snapshot_indices, recalls_rf, 'red', marker='^', linestyle='--', label='RandomForest Recall')
plt.plot(snapshot_indices, f1s_xgb, 'purple', marker='d', linestyle='--', label='XGBoost F1')
plt.plot(snapshot_indices, f1s_rf, 'brown', marker='x', linestyle='--', label='RandomForest F1')

plt.xlabel('Snapshot Index')
plt.ylabel('Metric')
plt.title('📊 Model Comparison: XGBoost vs Random Forest Over Time')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

# Evaluate XGBoost
y_pred_xgb, y_pred_prob_xgb, cm_xgb = evaluate_model_global(xgb_model, X_test, y_test, "XGBoost")

# Evaluate Random Forest
y_pred_rf, y_pred_prob_rf, cm_rf = evaluate_model_global(rf_model, X_test, y_test, "Random Forest")



summary = pd.DataFrame({
    "Model": ["XGBoost", "Random Forest"],
    "AUC": [
        roc_auc_score(y_test, y_pred_prob_xgb),
        roc_auc_score(y_test, y_pred_prob_rf)
    ],
    "Recall": [
        recall_score(y_test, y_pred_xgb),
        recall_score(y_test, y_pred_rf)
    ],
    "Precision": [
        classification_report(y_test, y_pred_xgb, output_dict=True)['1']['precision'],
        classification_report(y_test, y_pred_rf, output_dict=True)['1']['precision']
    ],
    "F1": [
        classification_report(y_test, y_pred_xgb, output_dict=True)['1']['f1-score'],
        classification_report(y_test, y_pred_rf, output_dict=True)['1']['f1-score']
    ]
})

print("\n📋 Global Model Comparison Summary:")
display(summary)



###importance

In [ ]:
def plot_feature_importance(model, X_train, model_name="Model", top_n=20):
    """
    Plot feature importance for tree-based models (XGBoost, RandomForest).
    """
    import pandas as pd
    import matplotlib.pyplot as plt

    # Handle model type
    if hasattr(model, "get_booster"):  # XGBoost
        importance = model.get_booster().get_score(importance_type='gain')
        fi = pd.DataFrame({
            'Feature': list(importance.keys()),
            'Importance': list(importance.values())
        })
    elif hasattr(model, "feature_importances_"):  # RandomForest
        fi = pd.DataFrame({
            'Feature': X_train.columns,
            'Importance': model.feature_importances_
        })
    else:
        raise ValueError(f"{model_name} does not support feature importance extraction.")

    # Sort and plot
    fi = fi.sort_values(by='Importance', ascending=False)
    plt.figure(figsize=(10,6))
    plt.barh(fi['Feature'][:top_n][::-1], fi['Importance'][:top_n][::-1])
    plt.title(f'📊 {model_name} Feature Importance (Top {top_n})')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.grid(alpha=0.4)
    plt.tight_layout()
    plt.show()

    return fi


In [ ]:
fi_xgb = plot_feature_importance(xgb_model, X_train, "XGBoost")
fi_rf = plot_feature_importance(rf_model, X_train, "Random Forest")


###Entropy Importance

In [ ]:
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import entropy

# ==============================================================
# 0️⃣ Sample test users for SHAP analysis (from test_df)
# ==============================================================

fraud_users = test_df[test_df["label"] == 1]["phone_no_m"].unique()
nonfraud_users = test_df[test_df["label"] == 0]["phone_no_m"].unique()

# sample subset of users (tune 200/800 as needed)
sampled_users = np.concatenate([
    np.random.choice(fraud_users, size=min(len(fraud_users), 200), replace=False),
    np.random.choice(nonfraud_users, size=min(len(nonfraud_users), 800), replace=False)
])

# build mask from test_df
sample_mask = test_df["phone_no_m"].isin(sampled_users)
X_sample = X_test.loc[sample_mask].copy()
y_sample = y_test.loc[sample_mask].copy()

print(f"✅ Sampled {len(X_sample)} snapshots from {len(sampled_users)} users "
      f"({(y_sample==1).sum()} fraud, {(y_sample==0).sum()} normal)")

# ==============================================================
# 1️⃣ Define prediction wrapper for SHAP
# ==============================================================

def model_predict(X):
    """Ensure SHAP calls the model correctly with the same features."""
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X, columns=X_sample.columns)
    return xgb_model.predict_proba(X)[:, 1]

# ==============================================================
# 2️⃣ Compute SHAP values for sampled subset
# ==============================================================

print("\n🧠 Computing SHAP values on sampled subset...")
explainer = shap.Explainer(model_predict, X_sample)
shap_values = explainer(X_sample)
shap_array = shap_values.values  # numpy array (n_samples, n_features)

# ==============================================================
# 3️⃣ Compute SHAP Entropy (fraud subset only)
# ==============================================================

fraud_mask = (y_sample == 1)
abs_shap = np.abs(shap_array[fraud_mask])
norm_shap = abs_shap / (abs_shap.sum(axis=1, keepdims=True) + 1e-9)
shap_entropy = entropy(norm_shap.T, base=2, axis=1)

entropy_df = (
    pd.DataFrame({
        "Feature": X_sample.columns,
        "SHAP_Entropy": shap_entropy
    })
    .sort_values("SHAP_Entropy", ascending=False)
    .reset_index(drop=True)
)

print("\n📋 Top 10 high-entropy (less consistent) features — fraud samples only:")
display(entropy_df.head(10))

# ==============================================================
# 4️⃣ Compare with XGBoost Gain Importance
# ==============================================================

gain_imp = xgb_model.get_booster().get_score(importance_type='gain')
gain_df = pd.DataFrame(list(gain_imp.items()), columns=["Feature", "Gain"])
merged = gain_df.merge(entropy_df, on="Feature", how="inner")

plt.figure(figsize=(8,6))
plt.scatter(merged["Gain"], merged["SHAP_Entropy"], alpha=0.7, color="teal")
plt.xlabel("XGBoost Gain (information value)")
plt.ylabel("SHAP Entropy (consistency)")
plt.title("📊 Gain vs SHAP Entropy (fraud class subset)")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# ==============================================================
# 5️⃣ Stable = high gain + low entropy
# ==============================================================

stable_features = merged.sort_values(["Gain", "SHAP_Entropy"], ascending=[False, True])
print("\n🏆 Top stable fraud predictors (high gain, low entropy):")
display(stable_features.head(10))


##Timeline based Model

####Generate timeline

###Load

In [ ]:
def load_raw_datasets(config):
    import os
    import pandas as pd

    if "ML" in config and "Events" in config["ML"]:
        events_cfg = config["ML"]["Events"]
    else:
        events_cfg = config["Events"]

    base = events_cfg["base_path"]
    files = events_cfg["files"]

    # --- Load all datasets ---
    df_voc = pd.read_csv(os.path.join(base, files["voc"]))
    df_sms = pd.read_csv(os.path.join(base, files["sms"]))
    df_app = pd.read_csv(os.path.join(base, files["app"]))
    df_user = pd.read_csv(os.path.join(base, files["user"]))

    # --- Normalize timestamps and add source column ---
    for df, src in [(df_voc, "VOC"), (df_sms, "SMS"), (df_app, "APP"), (df_user, "USER")]:
        df["source"] = src
        ts_col = [c for c in df.columns if "time" in c.lower()][0]
        df.rename(columns={ts_col: "event_time"}, inplace=True)
        df["event_time"] = pd.to_datetime(df["event_time"], errors="coerce")

    print("✅ Raw datasets loaded and timestamp-normalized.")
    return df_voc, df_sms, df_app, df_user

df_voc, df_sms, df_app, df_user = load_raw_datasets(config)


###Define user sequence

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

def make_user_sequences(events, feature_cols=None, max_seq_len=100):
    """
    Build per-user sequences for LSTM models.
    Each user's events are sorted by time and padded/truncated to fixed length.

    Parameters
    ----------
    events : pd.DataFrame
        Combined event dataset (all sources).
    feature_cols : list or None
        List of numeric columns to include as features.
        If None, uses all numeric columns except 'label'.
    max_seq_len : int
        Sequence length to pad/truncate to.

    Returns
    -------
    X_seq : np.ndarray
        Array of shape (n_users, max_seq_len, n_features)
    y : np.ndarray
        Array of shape (n_users,)
    users : list
        List of user IDs
    """
    events = events.copy()
    users, X_seq, y = [], [], []

    # 🔹 Encode categorical 'source' column numerically
    le = LabelEncoder()
    events["source_id"] = le.fit_transform(events["source"].astype(str))

    # 🔹 Determine feature columns
    if feature_cols is None:
        feature_cols = events.select_dtypes(include=["number"]).columns.difference(["label"]).tolist()
    if "source_id" not in feature_cols:
        feature_cols.append("source_id")

    print(f"\n📦 Using {len(feature_cols)} features: {feature_cols}")


    # ✅ Build per-user sequences
    for user, df_u in events.groupby("phone_no_m"):
        df_u = df_u.sort_values("event_time")

        feats = df_u[feature_cols].to_numpy(dtype=float)

        # Pad or truncate
        if len(feats) < max_seq_len:
            feats = np.pad(feats, ((max_seq_len - len(feats), 0), (0, 0)))
        else:
            feats = feats[-max_seq_len:]

        # User label = any fraud event → fraud
        label = int(df_u["label"].max())

        X_seq.append(feats)
        y.append(label)
        users.append(user)

    print(f"\n✅ Created sequences for {len(users)} users")
    print(f"   Fraud users: {sum(y)} ({np.mean(y)*100:.2f}%)")
    print(f"   Normal users: {len(y) - sum(y)} ({(1 - np.mean(y))*100:.2f}%)")

    X_seq = np.array(X_seq)
    y = np.array(y)

    print(f"\n📐 Final tensor shape: X={X_seq.shape}, y={y.shape}")
    return X_seq, y, users


###Build timeline (events)

In [ ]:
def merge_and_prepare_events(df_voc, df_sms, df_app, df_user):

    # --- 1️⃣ Normalize USER dataset ---
    if 'label' not in df_user.columns:
        raise KeyError("❌ 'label' column not found in user dataset")

    # Ensure numeric consistency
    df_user['label'] = df_user['label'].fillna(0).astype(int)
    df_user['idcard_cnt'] = df_user['idcard_cnt'].fillna(0).astype(float)
    df_user['arpu_value'] = df_user['arpu_value'].fillna(0).astype(float)

    # --- 2️⃣ Extract static info for merging (label + sim count only) ---
    static_user_info = df_user.groupby("phone_no_m", as_index=False)[["label", "idcard_cnt"]].max()

    # --- 3️⃣ Merge static info into other event tables ---
    df_voc = df_voc.merge(static_user_info, on="phone_no_m", how="left")
    df_sms = df_sms.merge(static_user_info, on="phone_no_m", how="left")
    df_app = df_app.merge(static_user_info, on="phone_no_m", how="left")


    # --- 4️⃣ Combine all transactional event sources ---
    # include df_user itself since arpu_value is event-like
    #events = pd.concat([df_voc, df_sms, df_app, df_user], ignore_index=True)
    # ✅ Keep only transactional events (VOC + SMS)
    #Drop app and user fee
    events = pd.concat([df_voc, df_sms], ignore_index=True)

    # --- 5️⃣ Fill and order ---
    events["label"] = events["label"].fillna(0).astype(int)
    events["event_time"] = pd.to_datetime(events["event_time"], errors="coerce")
    events = events.sort_values(["phone_no_m", "event_time"]).reset_index(drop=True)

    # --- 6️⃣ Summary ---
    print("\n🔎 Feature Summary per Source:")
    for src, df in [("VOC", df_voc), ("SMS", df_sms), ("APP", df_app), ("USER", df_user)]:
        print(f"\n📂 Source: {src}")
        print(f"   Events: {len(df):,}")
        print(f"   Users : {df['phone_no_m'].nunique():,}")
        print(f"   Columns ({len(df.columns)}): {', '.join(df.columns)}")

    print("\n📊 Combined Dataset Summary:")
    print(f"   Total events: {len(events):,}")
    print(f"   Unique users: {events['phone_no_m'].nunique():,}")
    print(f"   Fraud ratio: {events['label'].mean()*100:.2f}%")

    return events

events = merge_and_prepare_events(df_voc, df_sms, df_app, df_user)


###Split

In [ ]:


# ======================================
# 0️⃣ Clean Numeric Columns
# ======================================
events = events.copy()
numeric_cols = events.select_dtypes(include=["number"]).columns.difference(["label"])

# Replace NaN with 0 for numeric fields (avoids scaling issues)
events[numeric_cols] = events[numeric_cols].fillna(0)

print(f"\n📊 Numeric columns to scale ({len(numeric_cols)}): {numeric_cols.tolist()}")

# ======================================
# 1️⃣ Scale Numeric Features
# ======================================
scaler_seq = StandardScaler()
events[numeric_cols] = scaler_seq.fit_transform(events[numeric_cols])
print(f"📏 Scaled {len(numeric_cols)} numeric columns for event-level modeling.")

# ======================================
# 2️⃣ Create Train/Test User Split (if not exists)
# ======================================
split_dir = "splits"
train_split_file = f"{split_dir}/train_users.csv"
test_split_file = f"{split_dir}/test_users.csv"

os.makedirs(split_dir, exist_ok=True)

if os.path.exists(train_split_file) and os.path.exists(test_split_file):
    print("📂 Using existing user split from file...")
    train_users = set(pd.read_csv(train_split_file)["phone_no_m"])
    test_users  = set(pd.read_csv(test_split_file)["phone_no_m"])
else:
    print("🆕 Creating new unified user split (for LSTM)...")

    # One label per user
    user_labels = events.groupby("phone_no_m")["label"].max()
    fraud_users = user_labels[user_labels == 1].index
    normal_users = user_labels[user_labels == 0].index

    fraud_train, fraud_test = train_test_split(fraud_users, test_size=0.2, random_state=42)
    normal_train, normal_test = train_test_split(normal_users, test_size=0.2, random_state=42)

    train_users = set(fraud_train) | set(normal_train)
    test_users  = set(fraud_test)  | set(normal_test)

    pd.DataFrame({"phone_no_m": sorted(train_users)}).to_csv(train_split_file, index=False)
    pd.DataFrame({"phone_no_m": sorted(test_users)}).to_csv(test_split_file, index=False)
    print(f"✅ Saved user split to '{split_dir}/'")

print(f"✅ Train users: {len(train_users)} | Test users: {len(test_users)}")

# ======================================
# 3️⃣ Apply Split to Events
# ======================================
train_events = events[events["phone_no_m"].isin(train_users)]
test_events  = events[events["phone_no_m"].isin(test_users)]

# Sanity checks
assert len(set(train_events["phone_no_m"]) & set(test_events["phone_no_m"])) == 0, "❌ User leakage detected!"
assert train_events["label"].nunique() == 2, "❌ Training set must contain both classes"
assert test_events["label"].nunique() == 2, "❌ Test set must contain both classes"

# ======================================
# 4️⃣ Create Sequences (using multiple features)
# ======================================
numeric_features = [c for c in numeric_cols if c not in ["label"]]  # exclude label
max_seq_len=100
print(f"\n📦 Features used for sequences: {numeric_features}")
X_train, y_train, users_train = make_user_sequences(train_events, feature_cols=numeric_features, max_seq_len=max_seq_len)
X_test, y_test, users_test = make_user_sequences(test_events, feature_cols=numeric_features, max_seq_len=max_seq_len)

print("\n✅ Sequence Summary (per-user sequences):")
print(f"   X_train: {X_train.shape} | Fraud ratio: {np.mean(y_train)*100:.2f}%")
print(f"   X_test : {X_test.shape} | Fraud ratio: {np.mean(y_test)*100:.2f}%")

# ======================================
# 5️⃣ Consistency Check
# ======================================
rf_train = set(pd.read_csv(train_split_file)["phone_no_m"])
rf_test  = set(pd.read_csv(test_split_file)["phone_no_m"])
assert rf_train == train_users, "❌ Train user mismatch between LSTM and RF/XGB!"
assert rf_test  == test_users,  "❌ Test user mismatch between LSTM and RF/XGB!"
print("\n🔒 Consistency Check: ✅ Same users used for all models (LSTM, RF, XGBoost).")


###Cor

In [ ]:
# --- LSTM correlation ---
print("📊 Correlation Matrix — Raw Event Features (LSTM)")

corr_lstm = pd.DataFrame(events[numeric_cols]).corr()

plt.figure(figsize=(12,8))
sns.heatmap(corr_lstm, cmap='coolwarm', annot=False, fmt=".2f", linewidths=0.5)
plt.title("📊 Feature Correlation Heatmap — Raw Event Data (LSTM)")
plt.show()

# Optional: Highly correlated pairs
threshold = 0.85
corr_pairs_lstm = (
    corr_lstm.where(np.triu(np.ones(corr_lstm.shape), k=1).astype(bool))
    .stack()
    .reset_index()
)
corr_pairs_lstm.columns = ["Feature1", "Feature2", "Correlation"]
high_corr_lstm = corr_pairs_lstm[corr_pairs_lstm["Correlation"].abs() > threshold]
display(high_corr_lstm)


##F1

In [ ]:
from tensorflow.keras import backend as K

def f1_metric(y_true, y_pred):
    # Convert both tensors to float32 before math operations
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(K.round(y_pred), 'float32')

    tp = K.sum(y_true * y_pred)
    fp = K.sum((1 - y_true) * y_pred)
    fn = K.sum(y_true * (1 - y_pred))

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))


###Model

In [ ]:

weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(weights))
print(class_weights)

# ======================================
# 4️⃣ Build and train LSTM model
# ======================================

lstm_model = Sequential([
    Masking(mask_value=0.0, input_shape=(max_seq_len, X_train.shape[2])),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])


lstm_model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(1e-3),
    metrics=['AUC', 'Recall',f1_metric]
)

print("🚀 Training LSTM...")
lstm_history = lstm_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10, batch_size=64,
    class_weight=class_weights

)


###Test

In [ ]:

# Predict probabilities and labels
y_pred_prob = lstm_model.predict(X_test).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)


###Report

In [ ]:

def evaluate_lstm_global(model, X_test, y_test, model_name="LSTM"):
    """
    Evaluate trained LSTM on test sequences.
    Displays classification report and confusion matrix.
    """
    # Predict probabilities and labels
    y_pred_prob = model.predict(X_test).ravel()
    y_pred = (y_pred_prob > 0.5).astype(int)

    # Compute metrics
    auc = roc_auc_score(y_test, y_pred_prob)
    recall = recall_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, digits=4)
    cm = confusion_matrix(y_test, y_pred)

    # Print metrics
    print(f"\n📊 Classification Report — {model_name}")
    print(report)
    print(f"AUC: {auc:.4f} | Recall: {recall:.4f}")

    # Confusion Matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Normal (0)", "Fraud (1)"])
    disp.plot(cmap="Purples")
    plt.title(f"Confusion Matrix — {model_name}")
    plt.grid(False)
    plt.show()

    return {
        "model": model_name,
        "AUC": auc,
        "Recall": recall,
        "Precision": classification_report(y_test, y_pred, output_dict=True)['1']['precision'],
        "F1": classification_report(y_test, y_pred, output_dict=True)['1']['f1-score']
    }
lstm_results = evaluate_lstm_global(lstm_model, X_test, y_test, "LSTM")
lstm_results["Model"] = lstm_results.pop("model")


In [ ]:


plt.figure(figsize=(9,4))
plt.plot(lstm_history.history['AUC'], label='Train AUC')
plt.plot(lstm_history.history['val_AUC'], label='Val AUC')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.title('📈 LSTM AUC Over Epochs')
plt.legend()
plt.grid(True, alpha=0.4)
plt.show()
# Append LSTM to summary comparison
# Completely reset (delete all data)
plt.figure(figsize=(9,4))
plt.plot(lstm_history.history['f1_metric'], label='Train F1', color='purple')
plt.plot(lstm_history.history['val_f1_metric'], label='Val F1', color='magenta')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.title('📈 LSTM F1 Over Epochs')
plt.legend()
plt.grid(True, alpha=0.4)
plt.tight_layout()
plt.show()


summary = pd.concat([summary, pd.DataFrame([lstm_results])], ignore_index=True)
display(summary)


#freeze

In [ ]:
%pip freeze > "{project_path}requirement/freez/ML_requirements-lock.txt"
